In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import time
import sqlite3

## EXTREACT TRANSFORM AND LOAD HEAVY CONSTRUCTION EQUIPMENT DATA FROM TOP MANUFACTURERS

### HEAVY CONSTRUCTION EQUIPMENT DATA FROM HYUNDAI 

### EXPLORING HYUNDAI DATA:

In [2]:
url = urlopen("https://na.hd-hyundaice.com/equipment")
bsobj = soup(url.read())

In [3]:
bsobj.find_all("a")

[<a class="transition-colors duration-300 hover:text-gray-600" href="/contact-us" rel="" target="_self">Contact Us</a>,
 <a class="transition-colors duration-300 hover:text-gray-600" href="/request-quote" rel="" target="_self">Request a Quote</a>,
 <a class="transition-colors duration-300 hover:text-gray-600" href="/services-support/special-offers" rel="" target="_self">Current Offers</a>,
 <a class="transition-colors duration-300 hover:text-gray-600" href="/services-support/dealer-resources" rel="" target="_self">Dealer Resources</a>,
 <a class="transition-colors duration-300 hover:text-gray-600" href="https://ethics.hd.com/HtmlE/main.html" rel="noopener noreferrer" target="_blank">Corporate Ethics</a>,
 <a class="-m-1.5 p-1.5" data-gtm-name="header-logo" href="/"><span class="sr-only">Hyundai Construction Equipment</span><div class="-ml-4 w-48 md:ml-0 md:w-60"><svg class="w-full" height="65" viewbox="0 0 1775 260.9" width="43.382" xmlns="http://www.w3.org/2000/svg"><g data-name="Laye

In [4]:
for link in bsobj.find_all("a"):
    if "href" in link.attrs:
        print(link.attrs['href'])
        print(link.get_text())
        print("")

/contact-us
Contact Us

/request-quote
Request a Quote

/services-support/special-offers
Current Offers

/services-support/dealer-resources
Dealer Resources

https://ethics.hd.com/HtmlE/main.html
Corporate Ethics

/
Hyundai Construction Equipment

/newsroom
Newsroom

/find-dealer
Find a Dealer

/
Home


Equipment

/equipment/compact-excavators
Compact ExcavatorsSample Subhead

/equipment/compact-excavators/hx40a
HX40A

/equipment/compact-excavators/hx60a
HX60A

/equipment/compact-excavators/hx85a
HX85A

/equipment/compact-excavators/hx90a
HX90A

/equipment/compact-excavators/hx35az
HX35AZ

/equipment/compact-excavators/hx48az
HX48AZ

/equipment/compact-excavators/r17z-9a
R17Z-9A

/equipment/compact-excavators/r25z-9ak
R25Z-9AK

/equipment/compact-excavators/r30z-9ak
R30Z-9AK

/equipment/compact-excavators/r35z-9a
R35Z-9A

/equipment/compact-excavators/r55-9a
R55-9A

/equipment/compact-excavators/r60cr-9a
R60CR-9A

/equipment/compact-excavators/r80cr-9a
R80CR-9A

/equipment/compact-exca

for a in bsobj.find('div', {'class': 'container-max relative z-10'}).find_all('a', {'class': 'underline hover:text-black'}):
    print(a['href'])
    print(a.get_text())
    print("")

In [5]:
for a in bsobj.find('div', {'class': 'container-max relative z-10'}).find_all('a', {'class': 'underline hover:text-black'}):
    print(a.attrs.get('href', 'No href found'))  # Safe access
    print("")


/equipment/compact-excavators/hx40a

/equipment/compact-excavators/hx60a

/equipment/compact-excavators/hx85a

/equipment/compact-excavators/hx90a

/equipment/compact-excavators/hx35az

/equipment/compact-excavators/hx48az

/equipment/compact-excavators/r17z-9a

/equipment/compact-excavators/r25z-9ak

/equipment/compact-excavators/r30z-9ak

/equipment/compact-excavators/r35z-9a

/equipment/compact-excavators/r55-9a

/equipment/compact-excavators/r60cr-9a

/equipment/compact-excavators/r80cr-9a

/equipment/compact-excavators/hx55acr

/equipment/compact-excavators/hx27az

/equipment/compact-excavators/hx17az

/equipment/crawler-excavators/hx130a-lcr

/equipment/crawler-excavators/hx140a-l

/equipment/crawler-excavators/hx145a-lcr

/equipment/crawler-excavators/hx180a-l

/equipment/crawler-excavators/hx210a-l

/equipment/crawler-excavators/hx220a-l

/equipment/crawler-excavators/hx220a-lr

/equipment/crawler-excavators/hx235a-lcr

/equipment/crawler-excavators/hx260a-l

/equipment/crawler

## Extracting, Transforming and Loading Hyundai Heavy Construction Equipment Data:

In [7]:
base_url = "https://na.hd-hyundaice.com/"

In [8]:
# Open the main equipment page
response = urlopen(base_url + "/equipment")
html = response.read().decode('utf-8')  # Ensure UTF-8 decoding
bsobj = soup(html, "html.parser")

In [9]:
# Find all equipment links
equipment_links = [
    base_url + a.attrs['href']
    for a in bsobj.find('div', {'class': 'container-max relative z-10'}).find_all('a', {'class': 'underline hover:text-black'})
    if 'href' in a.attrs
]

In [10]:
# List to store all equipment
data = []

In [11]:
# Loop through each equipment link
for link in equipment_links:
    try:
        # Open the equipment detail page
        page = urlopen(link)
        equipment_html = page.read().decode('utf-8')  # Decode properly
        equipment_soup = soup(equipment_html, "html.parser")

        # Extract the equipment type (category)
        name_tags = equipment_soup.find_all("span", {
            "class": "flex-1 truncate border-b border-gray-200 py-1 text-xs text-gray-600 transition-all duration-300 group-last:text-black group-last-of-type:border-transparent group-hover/breadcrumb:border-green-600 group-hover/breadcrumb:text-black group-hover/breadcrumb:group-last:border-green-600 group-hover/breadcrumb:group-last-of-type:border-b dark:text-white group-last:dark:text-white group-hover/breadcrumb:dark:text-white"
        })

        # Ensure there are at least 2 span elements (one for type, one for model)
        if len(name_tags) >= 2:
            equipment_type = name_tags[-2].get_text(strip=True)  # Second last <span>
            model_name = name_tags[-1].get_text(strip=True)  # Last <span>
        else:
            equipment_type = "N/A"
            model_name = "N/A"

        # Extract specifications
        specs = {}
        spec_table = equipment_soup.find("table", {"class": "w-full table-fixed"})

        if spec_table:
            for row in spec_table.find_all("tr", {"class": "w-full border-b border-gray-200"}):
                cols = row.find_all("td")
                if len(cols) == 2:
                    key = cols[0].get_text(strip=True)  # First column is spec name
                    value = cols[1].get_text(strip=True)  # Second column is spec value
                    specs[key] = value

        # Extract breaker data (if no specification table is found)
        if not spec_table:
            breaker_specs = equipment_soup.find("div", {"class": "flex flex-col gap-5 dark:text-white md:flex-row md:gap-10"})
            if breaker_specs:
                for div in breaker_specs.find_all("div", {"class": "flex flex-col gap-2"}):
                    key_div = div.find("div", {"class": "text-xs text-white sm:text-sm md:text-white"})
                    value_div = div.find("div", {"class": "text-lg font-bold text-white sm:text-xl"})
                    if key_div and value_div:
                        key = key_div.get_text(strip=True)
                        value = value_div.get_text(strip=True)
                        specs[key] = value

        # Append to data list
        data.append({"Equipment Type": equipment_type, "Model": model_name, "Link": link, **specs})

        print(f"Scraped: {equipment_type} - {model_name}")  # Progress update

        # Sleep to avoid being blocked
        time.sleep(1)

    except Exception as e:
        print(f"Failed to scrape {link}: {e}")

# Convert to DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv("hyundai_equipment_data.csv", index=False)

print("Scraping complete. Data saved to equipment_data.csv")


Scraped: Compact Excavators - HX40A
Scraped: Compact Excavators - HX60A
Scraped: Compact Excavators - HX85A
Scraped: Compact Excavators - HX90A
Scraped: Compact Excavators - HX35AZ
Scraped: Compact Excavators - HX48AZ
Scraped: Compact Excavators - R17Z-9A
Scraped: Compact Excavators - R25Z-9AK
Scraped: Compact Excavators - R30Z-9AK
Scraped: Compact Excavators - R35Z-9A
Scraped: Compact Excavators - R55-9A
Scraped: Compact Excavators - R60CR-9A
Scraped: Compact Excavators - R80CR-9A
Scraped: Compact Excavators - HX55ACR
Scraped: Compact Excavators - HX27AZ
Scraped: Compact Excavators - HX17AZ
Scraped: Crawler Excavators - HX130A LCR
Scraped: Crawler Excavators - HX140A L
Scraped: Crawler Excavators - HX145A LCR
Scraped: Crawler Excavators - HX180A L
Scraped: Crawler Excavators - HX210A L
Scraped: Crawler Excavators - HX220A L
Scraped: Crawler Excavators - HX220A LR
Scraped: Crawler Excavators - HX235A LCR
Scraped: Crawler Excavators - HX260A L
Scraped: Crawler Excavators - HX300A L
Scra

In [ ]:
# List to store extracted data
hyundai_equipment_data = []
all_spec_keys = set()  # To track all specification names for dynamic columns

# Loop through each equipment link
for link in equipment_links:
    try:
        # Open the equipment detail page
        page = urlopen(link)
        equipment_html = page.read().decode('utf-8')  # Decode properly
        equipment_soup = soup(equipment_html, "html.parser")

        # Extract the equipment type and model name
        name_tags = equipment_soup.find_all("span", {
            "class": "flex-1 truncate border-b border-gray-200 py-1 text-xs text-gray-600 transition-all duration-300 group-last:text-black group-last-of-type:border-transparent group-hover/breadcrumb:border-green-600 group-hover/breadcrumb:text-black group-hover/breadcrumb:group-last:border-green-600 group-hover/breadcrumb:group-last-of-type:border-b dark:text-white group-last:dark:text-white group-hover/breadcrumb:dark:text-white"
        })

        if len(name_tags) >= 2:
            equipment_type = name_tags[-2].get_text(strip=True)
            model_name = name_tags[-1].get_text(strip=True)
        else:
            equipment_type = "N/A"
            model_name = "N/A"

        # Extract specifications (for machines)
        specs = {}
        spec_table = equipment_soup.find("table", {"class": "w-full table-fixed"})

        if spec_table:
            for row in spec_table.find_all("tr", {"class": "w-full border-b border-gray-200"}):
                cols = row.find_all("td")
                if len(cols) == 2:
                    key = cols[0].get_text(strip=True)
                    value = cols[1].get_text(strip=True).replace("\u2013", "–")  # Fix en dash
                    specs[key] = value
                    all_spec_keys.add(key)  # Track specification columns

        # Extract breaker data (if no specification table is found)
        if not spec_table:
            breaker_specs = equipment_soup.find("div", {"class": "flex flex-col gap-5 dark:text-white md:flex-row md:gap-10"})
            if breaker_specs:
                for div in breaker_specs.find_all("div", {"class": "flex flex-col gap-2"}):
                    key_div = div.find("div", {"class": "text-xs text-white sm:text-sm md:text-white"})
                    value_div = div.find("div", {"class": "text-lg font-bold text-white sm:text-xl"})
                    if key_div and value_div:
                        key = key_div.get_text(strip=True)
                        value = value_div.get_text(strip=True).replace("\u2013", "–")  # Fix en dash
                        specs[key] = value
                        all_spec_keys.add(key)  # Track breaker specifications

        # Store data
        hyundai_equipment_data.append({"Equipment Type": equipment_type, "Model": model_name, "Link": link, **specs})

        print(f"Scraped: {equipment_type} - {model_name}")  # Progress update

        # Sleep to avoid being blocked
        time.sleep(1)

    except Exception as e:
        print(f"Failed to scrape {link}: {e}")

# Convert to DataFrame
hyundai_df = pd.DataFrame(hyundai_equipment_data)

# Connect to SQLite database (or create if it doesn't exist)
conn = sqlite3.connect("equipment_data.db")
cursor = conn.cursor()

# Create table dynamically
column_definitions = ", ".join([f'"{col}" TEXT' for col in ["Equipment Type", "Model", "Link"] + list(all_spec_keys)])
cursor.execute(f'CREATE TABLE IF NOT EXISTS equipment_specs (id INTEGER PRIMARY KEY AUTOINCREMENT, {column_definitions})')

# Insert data into SQLite
for _, row in df.iterrows():
    columns = ", ".join(f'"{col}"' for col in row.index)
    placeholders = ", ".join("?" for _ in row.index)
    values = tuple(row.fillna("").values)  # Replace NaN with empty string
    cursor.execute(f'INSERT INTO equipment_specs ({columns}) VALUES ({placeholders})', values)

# Commit and close database
conn.commit()
conn.close()

print("Scraping complete. Data saved to SQLite database (equipment_data.db)")


Scraped: Compact Excavators - HX40A
Scraped: Compact Excavators - HX60A
Scraped: Compact Excavators - HX85A
Scraped: Compact Excavators - HX90A
Scraped: Compact Excavators - HX35AZ
Scraped: Compact Excavators - HX48AZ
Scraped: Compact Excavators - R17Z-9A
Scraped: Compact Excavators - R25Z-9AK
Scraped: Compact Excavators - R30Z-9AK
Scraped: Compact Excavators - R35Z-9A
Scraped: Compact Excavators - R55-9A
Scraped: Compact Excavators - R60CR-9A
Scraped: Compact Excavators - R80CR-9A
Scraped: Compact Excavators - HX55ACR
Scraped: Compact Excavators - HX27AZ
Scraped: Compact Excavators - HX17AZ
Scraped: Crawler Excavators - HX130A LCR
Scraped: Crawler Excavators - HX140A L
Scraped: Crawler Excavators - HX145A LCR
Scraped: Crawler Excavators - HX180A L
Scraped: Crawler Excavators - HX210A L
Scraped: Crawler Excavators - HX220A L
Scraped: Crawler Excavators - HX220A LR
Scraped: Crawler Excavators - HX235A LCR
Scraped: Crawler Excavators - HX260A L
Scraped: Crawler Excavators - HX300A L
Scra

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import time
import sqlite3

In [15]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd

# URL to scrape
Z_url = "https://en-product.zoomlion.com/product/pro_list.htm?sCat=57&fCat=&keywords="

# Request the page content
response = requests.get(Z_url)
Z_soup = soup(response.text, "html.parser")

# Extract equipment names
equipment_names = [div.text.strip() for div in Z_soup.find_all("div", class_="con con16")]

# Extract equipment models
models = [div.text.strip() for div in Z_soup.find_all("div", class_="tit tit22")]

# Extract specifications
Z_spec_data = []
for spec in Z_soup.find_all("div", class_="line line01 con16"):
    spec_title = spec.text.split(":")[0].strip()
    spec_value = spec.find("span", class_="num num01").text.strip() if spec.find("span", class_="num num01") else ""
    Z_spec_data.append((spec_title, spec_value))

# Organizing data into a structured format
Z_data = []
num_items = min(len(equipment_names), len(models))
for i in range(num_items):
    entry = {"Equipment Name": equipment_names[i], "Model": models[i]}
    for spec_title, spec_value in Z_spec_data:
        entry[spec_title] = spec_value
    data.append(entry)

# Convert to DataFrame
Z_df = pd.DataFrame(Z_data)

# Save to CSV
Z_df.to_csv("Zoomlion_equipment_data.csv", index=False)

print("Data successfully saved to Zoomlion_equipment_data.csv")


Data successfully saved to Zoomlion_equipment_data.csv


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape
url = "https://en-product.zoomlion.com/product/pro_list.htm?sCat=57&fCat=&keywords="

# Request the page content
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Extract equipment names
equipment_names = [div.text.strip() for div in soup.find_all("div", class_="con con16")]

# Extract equipment models
models = [div.text.strip() for div in soup.find_all("div", class_="tit tit22")]

# Extract specifications
data = []
containers = soup.find_all("div", class_="cont")
for container in containers:
    specs = container.find_all("div", class_="line line01 con16")
    spec_dict = {}
    for spec in specs:
        spec_title = spec.text.split(":")[0].strip()
        spec_value = spec.find("span", class_="num num01").text.strip() if spec.find("span", class_="num num01") else ""
        spec_dict[spec_title] = spec_value
    data.append(spec_dict)

# Organizing data into a structured format
num_items = min(len(equipment_names), len(models), len(data))
final_data = []
for i in range(num_items):
    entry = {"Equipment Name": equipment_names[i], "Model": models[i], **data[i]}
    final_data.append(entry)

# Convert to DataFrame
df = pd.DataFrame(final_data)

# Save to CSV
df.to_csv("Zoomlion_equipment_data.csv", index=False)

print("Data successfully saved to Zoomlion_equipment_data.csv")


Data successfully saved to Zoomlion_equipment_data.csv


In [17]:
print(f"Found {len(equipment_names)} equipment names")
print(f"Found {len(models)} models")
print(f"Found {len(containers)} spec containers")

for i, container in enumerate(containers[:5]):  # Check first few
    print(f"\nEquipment {i+1} Specifications:")
    print(container.prettify())  # Show full HTML


Found 0 equipment names
Found 0 models
Found 0 spec containers


In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape
url = "https://en-product.zoomlion.com/product/pro_list.htm?sCat=57&fCat=&keywords="

# Request the page content
response = requests.get(url)
if response.status_code == 200:
    print("Successfully accessed the URL.")
else:
    print(f"Failed to access the URL. Status code: {response.status_code}")

soup = BeautifulSoup(response.text, "html.parser")

# Extract equipment names
equipment_names = [div.text.strip() for div in soup.find_all("div", class_="con con16")]
print(f"Found {len(equipment_names)} equipment names.")

# Extract equipment models
models = [div.text.strip() for div in soup.find_all("div", class_="tit tit22")]
print(f"Found {len(models)} models.")

# Extract specifications
data = []
containers = soup.find_all("div", class_="cont")
print(f"Found {len(containers)} specification containers.")
for container in containers:
    specs = container.find_all("div", class_="line line01 con16")
    spec_dict = {}
    for spec in specs:
        spec_title = spec.text.split(":")[0].strip()
        spec_value = spec.find("span", class_="num num01").text.strip() if spec.find("span", class_="num num01") else ""
        spec_dict[spec_title] = spec_value
    data.append(spec_dict)

# Organizing data into a structured format
num_items = min(len(equipment_names), len(models), len(data))
final_data = []
for i in range(num_items):
    entry = {"Equipment Name": equipment_names[i], "Model": models[i], **data[i]}
    final_data.append(entry)

# Convert to DataFrame
df = pd.DataFrame(final_data)

# Save to CSV
df.to_csv("Zoomlion_equipment_data.csv", index=False)

print("Data successfully saved to Zoomlion_equipment_data.csv")


Successfully accessed the URL.
Found 0 equipment names.
Found 0 models.
Found 0 specification containers.
Data successfully saved to Zoomlion_equipment_data.csv


In [19]:
print(soup.prettify())  # View the full HTML structure

<!DOCTYPE html>
<html>
 <head>
  <title>
   Products-ZOOMLION
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
  <meta content="webkit" name="renderer"/>
  <link fixed="true" href="/resources/favicon.ico" rel="stylesheet" type="image/x-icon"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="design by www.jerei.com" name="author"/>
  <link href="/resources/ui/ui.css" rel="stylesheet"/>
  <link href="/resources/web/css/style.css" rel="stylesheet"/>
  <link href="/resources/web/css/responsive.css" rel="stylesheet"/>
  <link href="/resources/web/css/swiper.min.css" rel="stylesheet"/>
  <link href="/resources/web/css/animate.min.css" rel="stylesheet"/>
  <!-- 兼容ie8媒体查询以及视频播放 -->
  <!--[if lt IE 9]>
  <script src="/resources/web/lib/H5shiv.js"></script>
  <script src="/resources/web/js/css3-mediaqueries.js"> </script>
  <script src="/resources/web/js/ckplayer/ckplayer.